In [1]:
import yfinance as yf
import pandas as pd
import re
import time
import numpy as np
import holidays
from datetime import datetime, timedelta
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

In [2]:
tickers_db = pd.read_csv('earnings_calendar.csv')
tickers_db = tickers_db.dropna(subset=['Reported EPS'])
tickers_db = tickers_db.dropna(subset=['EPS Estimate'])
tickers_db = tickers_db.dropna(subset=['Surprise(%)'])
tickers_db = tickers_db.dropna(subset=['Earnings Date'])
tickers_db = tickers_db.dropna(subset=['Ticker'])
tickers_db

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Ticker
16,2024-06-12 16:00:00-04:00,10.84,10.96,0.0107,AVGO
17,2024-03-07 16:00:00-05:00,10.29,10.99,0.0678,AVGO
18,2023-12-07 16:00:00-05:00,10.98,11.06,0.0071,AVGO
19,2023-08-31 16:00:00-04:00,10.42,10.54,0.0112,AVGO
20,2023-06-01 16:00:00-04:00,10.08,10.32,0.0236,AVGO
...,...,...,...,...,...
6517,2023-08-08 07:00:00-04:00,1.32,1.41,0.0719,ZTS
6518,2023-05-04 07:00:00-04:00,1.26,1.31,0.0389,ZTS
6519,2023-02-14 07:00:00-05:00,1.05,1.15,0.0990,ZTS
6520,2022-11-03 07:00:00-04:00,1.24,1.21,-0.0207,ZTS


In [3]:
tickers_db

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Ticker
16,2024-06-12 16:00:00-04:00,10.84,10.96,0.0107,AVGO
17,2024-03-07 16:00:00-05:00,10.29,10.99,0.0678,AVGO
18,2023-12-07 16:00:00-05:00,10.98,11.06,0.0071,AVGO
19,2023-08-31 16:00:00-04:00,10.42,10.54,0.0112,AVGO
20,2023-06-01 16:00:00-04:00,10.08,10.32,0.0236,AVGO
...,...,...,...,...,...
6517,2023-08-08 07:00:00-04:00,1.32,1.41,0.0719,ZTS
6518,2023-05-04 07:00:00-04:00,1.26,1.31,0.0389,ZTS
6519,2023-02-14 07:00:00-05:00,1.05,1.15,0.0990,ZTS
6520,2022-11-03 07:00:00-04:00,1.24,1.21,-0.0207,ZTS


In [4]:
tickers_db['Earnings Date'] = pd.to_datetime(tickers_db['Earnings Date'], utc=True).dt.tz_localize(None)
tickers_db.reset_index(drop=True, inplace=True)

In [5]:
tickers_db.dtypes

Earnings Date    datetime64[ns]
EPS Estimate            float64
Reported EPS            float64
Surprise(%)             float64
Ticker                   object
dtype: object

In [6]:
def next_working_day(date):
    us_holidays = holidays.UnitedStates()
    next_day = date + timedelta(days=1)
    
    # Keep adding days until we find a weekday that is not a holiday
    while next_day.weekday() >= 5 or next_day in us_holidays:
        next_day += timedelta(days=1)
        
    return next_day

In [7]:
tickers_db['next_working_day'] = tickers_db['Earnings Date'].apply(next_working_day)

In [8]:
tickers_db

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%),Ticker,next_working_day
0,2024-06-12 20:00:00,10.84,10.96,0.0107,AVGO,2024-06-13 20:00:00
1,2024-03-07 21:00:00,10.29,10.99,0.0678,AVGO,2024-03-08 21:00:00
2,2023-12-07 21:00:00,10.98,11.06,0.0071,AVGO,2023-12-08 21:00:00
3,2023-08-31 20:00:00,10.42,10.54,0.0112,AVGO,2023-09-01 20:00:00
4,2023-06-01 20:00:00,10.08,10.32,0.0236,AVGO,2023-06-02 20:00:00
...,...,...,...,...,...,...
4322,2023-08-08 11:00:00,1.32,1.41,0.0719,ZTS,2023-08-09 11:00:00
4323,2023-05-04 11:00:00,1.26,1.31,0.0389,ZTS,2023-05-05 11:00:00
4324,2023-02-14 12:00:00,1.05,1.15,0.0990,ZTS,2023-02-15 12:00:00
4325,2022-11-03 11:00:00,1.24,1.21,-0.0207,ZTS,2022-11-04 11:00:00


In [9]:
# Your get_close_price function
def get_close_price(ticker, date):
    stock = yf.Ticker(ticker)
    start_date = datetime.strptime(date, '%Y-%m-%d')
    end_date = start_date + timedelta(days=1)
    hist = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    if hist.empty:
        return None
    close_price = hist['Close'][0]
    return close_price

# Function to apply to each row
def get_close_price_for_row(row):
    date_str = row['Earnings Date'].strftime('%Y-%m-%d')  # Convert Timestamp to string
    return get_close_price(row['Ticker'], date_str)

# Apply the function to the DataFrame
tickers_db['Close Earnings Day'] = tickers_db.apply(get_close_price_for_row, axis=1)

CVX: No price data found, symbol may be delisted (1d 2023-07-23 -> 2023-07-24)
CPRT: No price data found, symbol may be delisted (1d 2023-02-20 -> 2023-02-21)
EPAM: Data doesn't exist for startDate = 1723089600, endDate = 1723176000
NDSN: No price data found, symbol may be delisted (1d 2023-02-20 -> 2023-02-21)
RTX: Data doesn't exist for startDate = 1737435600, endDate = 1737522000
WMB: No price data found, symbol may be delisted (1d 2023-02-20 -> 2023-02-21)


In [10]:
def get_open_price(ticker, date):
    stock = yf.Ticker(ticker)
    start_date = datetime.strptime(date, '%Y-%m-%d')
    end_date = start_date + timedelta(days=1)
    hist = stock.history(start=start_date.strftime('%Y-%m-%d'), end=end_date.strftime('%Y-%m-%d'))
    if hist.empty:
        return None
    open_price = hist['Open'][0]
    return open_price

# Function to apply to each row
def get_open_price_for_row(row):
    date_str = row['next_working_day'].strftime('%Y-%m-%d')  # Convert Timestamp to string
    return get_open_price(row['Ticker'], date_str)

# Apply the function to the DataFrame
tickers_db['Open Earnings Day'] = tickers_db.apply(get_open_price_for_row, axis=1)

GILD: No price data found, symbol may be delisted (1d 2024-04-26 -> 2024-04-27)
STZ: No price data found, symbol may be delisted (1d 2023-04-07 -> 2023-04-08)
EPAM: Data doesn't exist for startDate = 1723176000, endDate = 1723262400
LW: No price data found, symbol may be delisted (1d 2023-04-07 -> 2023-04-08)
RTX: Data doesn't exist for startDate = 1737522000, endDate = 1737608400
WBA: No price data found, symbol may be delisted (1d 2024-03-29 -> 2024-03-30)


In [37]:
columns_order = [
    'Ticker','Earnings Date','next_working_day', 'EPS Estimate', 'Reported EPS', 'Surprise(%)', 
     'Close Earnings Day', 'Open Earnings Day'
]

tickers_db = tickers_db[columns_order]
tickers_db['Close Earnings Day'] = tickers_db['Close Earnings Day'].round(2)
tickers_db['Open Earnings Day'] = tickers_db['Open Earnings Day'].round(2)
tickers_db['Gap'] = tickers_db['Close Earnings Day'] - tickers_db['Open Earnings Day']
tickers_db['Gap']=tickers_db['Gap'].abs()
tickers_db['Gap'] = round(tickers_db['Gap'],2)
tickers_db['Gap_%'] = tickers_db['Gap']/tickers_db['Open Earnings Day']
tickers_db['Gap_%'] = round(tickers_db['Gap_%'],2)
tickers_db['cisne_negro'] = tickers_db['Gap_%'] > 0.1

In [38]:
tickers_db

,Ticker,Earnings Date,next_working_day,EPS Estimate,Reported EPS,Surprise(%),Close Earnings Day,Open Earnings Day,Gap,Gap_%,cisne_negro
0,AVGO,2024-06-12 20:00:00,2024-06-13 20:00:00,10.84,10.96,0.0107,149.55,171.00,21.45,0.13,True
1,AVGO,2024-03-07 21:00:00,2024-03-08 21:00:00,10.29,10.99,0.0678,140.70,139.43,1.27,0.01,False
2,AVGO,2023-12-07 21:00:00,2023-12-08 21:00:00,10.98,11.06,0.0071,92.23,92.78,0.55,0.01,False
3,AVGO,2023-08-31 20:00:00,2023-09-01 20:00:00,10.42,10.54,0.0112,92.29,90.19,2.10,0.02,False
4,AVGO,2023-06-01 20:00:00,2023-06-02 20:00:00,10.08,10.32,0.0236,79.00,79.06,0.06,0.00,False
...,...,...,...,...,...,...,...,...,...,...,...
4322,ZTS,2023-08-08 11:00:00,2023-08-09 11:00:00,1.32,1.41,0.0719,189.30,189.94,0.64,0.00,False
4323,ZTS,2023-05-04 11:00:00,2023-05-05 11:00:00,1.26,1.31,0.0389,178.47,180.00,1.53,0.01,False
4324,ZTS,2023-02-14 12:00:00,2023-02-15 12:00:00,1.05,1.15,0.0990,171.90,171.90,0.00,0.00,False
4325,ZTS,2022-11-03 11:00:00,2022-11-04 11:00:00,1.24,1.21,-0.0207,131.14,131.84,0.70,0.01,False


In [46]:
cisne_negro = tickers_db[tickers_db['cisne_negro'] == True]
print('De ' + str(len(tickers_db)) + ' Earnings, hay ' + str(len(cisne_negro)) + ' Cisnes Negro')
cisne_negro

De 4327 Earnings, hay 179 Cisnes Negro


,Ticker,Earnings Date,next_working_day,EPS Estimate,Reported EPS,Surprise(%),Close Earnings Day,Open Earnings Day,Gap,Gap_%,cisne_negro
0,AVGO,2024-06-12 20:00:00,2024-06-13 20:00:00,10.84,10.96,0.0107,149.55,171.00,21.45,0.13,True
16,DECK,2024-05-23 20:00:00,2024-05-24 20:00:00,2.89,4.95,0.7146,904.65,1014.92,110.27,0.11,True
18,DECK,2023-10-26 20:00:00,2023-10-27 20:00:00,4.43,6.82,0.5382,484.58,545.19,60.61,0.11,True
36,MO,2023-04-27 11:00:00,2023-04-28 11:00:00,1.18,1.18,-0.0038,42.28,47.47,5.19,0.11,True
37,MO,2023-02-01 12:00:00,2023-02-02 12:00:00,1.17,1.18,0.0043,41.70,47.36,5.66,0.12,True
...,...,...,...,...,...,...,...,...,...,...,...
3839,TTWO,2022-11-07 21:00:00,2022-11-08 21:00:00,1.37,1.30,-0.0540,108.40,91.90,16.50,0.18,True
3997,ULTA,2024-03-14 20:00:00,2024-03-15 20:00:00,7.53,8.08,0.0732,565.44,504.00,61.44,0.12,True
4000,ULTA,2023-05-25 20:00:00,2023-05-26 20:00:00,6.87,6.88,0.0015,485.12,436.11,49.01,0.11,True
4049,UHS,2023-02-27 21:00:00,2023-02-28 21:00:00,2.96,3.02,0.0205,146.04,128.41,17.63,0.14,True


In [45]:
tickers_db.to_csv('open_close_gaps.csv', index=False)